[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AMLA-UBC/100-Exploring-the-World-of-Modern-Machine-Learning/blob/main/Oneshot_Diffusion_Model_Finetuning_Tutorial.ipynb)

## DreamArtist: Controllable One-Shot Text-to-Image Generation via Contrastive Prompt-Tuning

DreamArtist offers a solution to the limitations of text-to-image generation models, which struggle with synthesizing diverse and high-quality images based on new concepts. DreamArtist uses a contrastive prompt-tuning learning method, which introduces both positive and negative embeddings as pseudo-words and trains them jointly. The positive embedding focuses on learning the characteristics of the reference image, while the negative embedding introspects in a self-supervised manner to correct any mistakes or inadequacies from the positive embedding. This results in the model learning not just what is correct, but also what to avoid. The DreamArtist has been tested and shown to generate high-quality images that also possess form, content, and context, and the pseudo-words have similar properties as true words in generating images.

<br>

Read more about DreamArtist [here](https://arxiv.org/abs/2211.11337).

<br>

If you are running on Gradient, use the [Gradient version](https://raw.githubusercontent.com/AMLA-UBC/100-Exploring-the-World-of-Modern-Machine-Learning/main/assets/sd_webui_gradient.txt).

In [ ]:
!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
!pip install -q --pre triton
!pip install -q --upgrade fastapi==0.90

!git clone -b v1.6 https://github.com/camenduru/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git /content/stable-diffusion-webui/extensions/DreamArtist
%cd /content/stable-diffusion-webui

!wget https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/anything-v4.0-pruned.ckpt
!wget https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned.ckpt
!wget https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/Dreamshaper_3.32_baked_vae_clip_fix.ckpt
!wget https://huggingface.co/claudfuen/photorealistic-fuen-v1/resolve/main/model.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/photorealistic-fuen-v1.ckpt
!wget https://huggingface.co/FredZhang7/paint-journey-v2/resolve/main/paint_journey_v2.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/paint_journey_v2.ckpt
!wget https://huggingface.co/FredZhang7/paint-journey-v2/resolve/main/paint_journey_v2.vae.pt -O /content/stable-diffusion-webui/models/Stable-diffusion/paint_journey_v2.vae.pt
!wget https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt -O /content/stable-diffusion-webui/models/Stable-diffusion/anything-v4.0-pruned.vae.pt

!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py

## Basic Training

[Source of information](https://github.com/7eu7d7/DreamArtist-stable-diffusion)

First create the positive and negative embeddings in DreamArtist Create Embedding Tab.
![](https://github.com/7eu7d7/DreamArtist-sd-webui-extension/raw/master/imgs/create.jpg)

After that, the `names` of the positive and negative embedding (`{name}` and `{name}-neg`) should be filled into the `txt2img` tab with some common descriptions. This will ensure a correct preview image. Here's an example with `{name}`=`test1`.

![](https://github.com/7eu7d7/DreamArtist-sd-webui-extension/raw/master/imgs/preview.png)

Then, select positive embedding and set the parameters and image folder path in the `DreamArtist Train` tab to start training. The corresponding negative embedding is loaded automatically. If your VRAM is low or you want save time, you can uncheck the `reconstruction`.

*better to train without filewords*

![](https://github.com/7eu7d7/DreamArtist-sd-webui-extension/raw/master/imgs/train.jpg)

Remember to check the option below, otherwise the preview is wrong.

![](https://github.com/7eu7d7/DreamArtist-sd-webui-extension/raw/master/imgs/fromtxt.png)

<br>

## Attention Mask
Attention Mask can strengthen or weaken the learning intensity of some local areas. 
Attention Mask is a grayscale image whose grayscale values are related to the learning intensity show in the following table.

| grayscale | 0% | 25% | 50%  | 75%  | 100% |
|-----------|----|-----|------|------|------|
| intensity | 0% | 50% | 100% | 300% | 500% |

The Attention Mask is in the same folder as the training image and its name is the name of the training image + "_att".
You can choose whether to enable Attention Mask for training.

![](https://github.com/7eu7d7/DreamArtist-sd-webui-extension/raw/master/imgs/att_map.jpg)

Since there is a self-attention operation in VAE, it may change the distribution of features. 
In the ***Process Att-Map*** tab, it can superimpose the attention map of self-attention on the original Att-Map.

<br>

## Dynamic CFG
Dynamic CFG can improve the performance, especially when the data set is large (>20). 
For example, linearly from 1.5 to 3.0 (1.5-3.0), or with a 0-π/2 cycle of cosine (1.5-3.0:cos), or with a -π/2-0 cycle of cosine (1.5-3.0:cos2).
Or you can also customize non-linear functions, such as 2.5-3.5:torch.sqrt(rate), where rate is a variable from 0-1.

<br>

## Inference
Fill the trained positive and negative embedding into txt2img to generate with DreamArtist prompt.

![](https://github.com/7eu7d7/DreamArtist-sd-webui-extension/raw/master/imgs/gen.jpg)



## Launch Camenduru (HuggingFace Automatic1111) WebUI with DreamArtist

In [ ]:
!python launch.py --share --xformers --enable-insecure-extension-access

## Add More Diffusion Models

In [ ]:
model_checkpoint = "nitrosocke/spider-verse-diffusion" #@param ["nitrosocke/spider-verse-diffusion", "nitrosocke/Nitro-Diffusion", "nitrosocke/Ghibli-Diffusion", "dallinmackay/Van-Gogh-diffusion", "coreco/seek.art_MEGA", "Aybeeceedee/knollingcase", "Fictiverse/Stable_Diffusion_VoxelArt_Model", "webui/stable-diffusion-2-1"] {allow-input: true}
URL_MAP = {
  "nitrosocke/spider-verse-diffusion": "https://huggingface.co/nitrosocke/spider-verse-diffusion/resolve/main/spiderverse-v1-pruned.ckpt",
  "nitrosocke/Nitro-Diffusion": "https://huggingface.co/nitrosocke/Nitro-Diffusion/resolve/main/nitroDiffusion-v1.ckpt",
  "nitrosocke/Ghibli-Diffusion": "https://huggingface.co/nitrosocke/Ghibli-Diffusion/resolve/main/ghibli-diffusion-v1.ckpt",
  "dallinmackay/Van-Gogh-diffusion": "https://huggingface.co/dallinmackay/Van-Gogh-diffusion/resolve/main/Van-Gogh-Style-lvngvncnt-v2.ckpt",
  "coreco/seek.art_MEGA": "https://huggingface.co/coreco/seek.art_MEGA/resolve/main/seek_art_mega_v1.ckpt",
  "Aybeeceedee/knollingcase": "https://huggingface.co/Aybeeceedee/knollingcase/resolve/main/knollingcase.ckpt",
  "Fictiverse/Stable_Diffusion_VoxelArt_Model": "https://huggingface.co/Fictiverse/Stable_Diffusion_VoxelArt_Model/resolve/main/VoxelArt_v1.ckpt",
  "webui/stable-diffusion-2-1": "https://huggingface.co/webui/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors"
}

checkpoint_name = model_checkpoint.split("/")[-1]
checkpoint_url = URL_MAP[model_checkpoint]

!wget {checkpoint_url} -O /content/stable-diffusion-webui/models/Stable-diffusion/{checkpoint_name}.ckpt